In [1]:
import io
import os
import platform
import pdb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.pyplot import cm

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import preprocessing

In [2]:
data=pd.read_csv('toronto.csv')

In [3]:
data.fillna(np.mean(data),inplace=True)

In [4]:
data.head()

,Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Wind Chill,NO,NO2,NOx,O3,PM25
0,-6.2,-13.2,58.0,25.0,43.0,16.1,100.68,-16.0,3,15,18,19,6
1,-6.0,-13.7,55.0,24.0,41.0,16.1,100.59,-16.0,2,14,16,19,7
2,-6.2,-13.3,57.0,24.0,45.0,16.1,100.56,-16.0,1,10,11,22,7
3,-6.0,-12.0,63.0,24.0,32.0,16.1,100.58,-15.0,1,10,11,21,7
4,-6.0,-12.2,62.0,24.0,35.0,16.1,100.50,-15.0,2,11,13,19,7


In [6]:
dataset=pd.DataFrame(data,columns=data.columns[:])

In [7]:
scaler = preprocessing.MinMaxScaler() 
scaled_values = scaler.fit_transform(dataset) 
dataset.loc[:,:] = scaled_values

In [8]:
data.head()

,Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Wind Chill,NO,NO2,NOx,O3,PM25
0,0.340037,0.334572,0.468354,0.685714,0.605634,1.0,0.457529,0.631579,3,15,18,19,6
1,0.343693,0.325279,0.430380,0.657143,0.577465,1.0,0.440154,0.631579,2,14,16,19,7
2,0.340037,0.332714,0.455696,0.657143,0.633803,1.0,0.434363,0.631579,1,10,11,22,7
3,0.343693,0.356877,0.531646,0.657143,0.450704,1.0,0.438224,0.657895,1,10,11,21,7
4,0.343693,0.353160,0.518987,0.657143,0.492958,1.0,0.422780,0.657895,2,11,13,19,7


In [9]:
data_set=np.asarray(dataset,dtype=np.float32)
seq_len=30 + 1
x=len(data_set)-seq_len
sequences = [data_set[t:t+seq_len] for t in range(x)]

In [10]:
len(sequences)

8729

In [11]:
seq=torch.FloatTensor(sequences)

In [12]:
seq.shape

torch.Size([8729, 31, 13])

In [14]:
split_row=round(0.90*seq.size(0))
x_train_set=seq[:split_row, :-1]
y_train_set=seq[:split_row, -1]
x_valid_set=seq[split_row:, :-1]
y_valid_set=seq[split_row:, -1]

In [15]:
class LSTM(nn.Module):
    def __init__(self,input_size,hidden_size,num_layers=1,dropout=0,bidirectional=False):
        super(LSTM,self).__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size
        self.num_layers=num_layers
        self.dropout=dropout
        self.bidirectional=bidirectional
        self.lstm = nn.LSTM(input_size,
                            hidden_size,
                            num_layers,
                            dropout=dropout,
                            bidirectional=bidirectional)
        self.linear = nn.Linear(hidden_size, 13)
        
    def forward(self,inputs,hidden):
        outputs,hidden=self.lstm(inputs,hidden)
        predictions=self.linear(outputs[-1])
        return predictions,outputs,hidden
    
    def init_hidden(self,batch_size):
        num_directions=2 if self.bidirectional else 1
        hidden = (torch.zeros(self.num_layers*num_directions, batch_size, self.hidden_size),
                  torch.zeros(self.num_layers*num_directions, batch_size, self.hidden_size))
        return hidden

In [16]:
def get_batch(x,y,i,batch_size):
    if x.dim() == 2:
        x = x.unsqueeze(2)
    batch_x = x[(i*batch_size):(i*batch_size)+batch_size, :, :]
    batch_y = y[(i*batch_size):(i*batch_size)+batch_size]

    # Reshape Tensors into (seq_len, batch_size, input_size) format for the LSTM.
    batch_x = batch_x.transpose(0, 1)
    
    return batch_x, batch_y

In [ ]:
input_size=13
hidden_size=24
num_layers=2
lstm=LSTM(input_size,hidden_size,num_layers)

learning_rate=0.01
max_grad_norm=5
loss_fn = nn.MSELoss()
optimizer = optim.Adam(lstm.parameters(), lr=learning_rate)

batch_size = 8
num_epochs = 20 #3
num_sequences = x_train_set.size(0)
num_batches = num_sequences //batch_size

print("Training model for {} epoch of {} batches".format(num_epochs, num_batches))
for epoch in range(num_epochs):
    total_loss = 0

    # Shuffle input and target sequences.
    idx = torch.randperm(x_train_set.size(0))
    x = x_train_set[idx]
    y = y_train_set[idx]

    for i in range(num_batches):
        # Get input and target batches and reshape for LSTM.
        batch_x, batch_y = get_batch(x_train_set, y_train_set, i, batch_size)

        # Reset the gradient.
        lstm.zero_grad()
        
        # Initialize the hidden states (see the function lstm.init_hidden(batch_size)).
        hidden = lstm.init_hidden(batch_size)
        
        # Complete a forward pass.
        y_pred, outputs, hidden = lstm(batch_x,hidden)
        
        # Calculate the loss with the 'loss_fn'.
        loss = loss_fn(y_pred,batch_y)
        
        # Compute the gradient.
        loss.backward()
        
        # Clip to the gradient to avoid exploding gradient.
        nn.utils.clip_grad_norm_(lstm.parameters(), max_grad_norm)

        # Make one step with optimizer.
        optimizer.step()
        
        # Accumulate the total loss.
        total_loss += loss.data

    print("Epoch {}: Loss = {:.8f}".format(epoch+1, total_loss/num_batches))

Training model for 20 epoch of 982 batches
Epoch 1: Loss = 0.01888363
Epoch 2: Loss = 0.01082577
Epoch 3: Loss = 0.00895088
Epoch 4: Loss = 0.00805519
Epoch 5: Loss = 0.00783074
Epoch 6: Loss = 0.00760383
Epoch 7: Loss = 0.00745616
Epoch 8: Loss = 0.00734614
Epoch 9: Loss = 0.00715117
Epoch 10: Loss = 0.00694480
Epoch 11: Loss = 0.00682934
Epoch 12: Loss = 0.00667298
Epoch 13: Loss = 0.00658471
